In [1]:
!pip install easyocr pdf2image



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 6.0 MB/s eta 0:00:00


In [2]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (266 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123614 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!unzip /content/batch1.zip

unzip:  cannot find or open /content/batch1.zip, /content/batch1.zip.zip or /content/batch1.zip.ZIP.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
#extract texts from the pdf

import easyocr
from pdf2image import convert_from_path
from PIL import Image
import os

# Path to the PDF file
pdf_directory = '/content/drive/MyDrive/লিখিত পরীক্ষার কক্ষ ভিত্তিক হাজিরা শীট/data entry control oparator'

image_dir = 'data entry'
os.makedirs(image_dir, exist_ok=True)
output_file = 'data entry.txt'
reader = easyocr.Reader(['en'])

with open(output_file, 'w', encoding='utf-8') as f:
    # Iterate through each PDF file in the directory
    for pdf_file in sorted(os.listdir(pdf_directory)):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, pdf_file)

            # Convert PDF to a list of images
            pages = convert_from_path(pdf_path)

            # Iterate through each page image
            for page_num, page_image in enumerate(pages):
                # Save the image for reference
                image_filename = f'{os.path.splitext(pdf_file)[0]}_page_{page_num + 1}.png'
                image_path = os.path.join(image_dir, image_filename)
                page_image.save(image_path, 'PNG')

                # Perform OCR on the image
                result = reader.readtext(image_path)

                # Write the OCR result to the output file
                f.write(f'{pdf_file} - Page {page_num + 1}\n')
                for (bbox, text, prob) in result:
                    f.write(text + '\n')
                f.write('\n' + '='*80 + '\n')

print(f'Text extracted and saved to {output_file}')
print(f'Images extracted and saved in {image_dir} directory')

Text extracted and saved to data entry.txt
Images extracted and saved in data entry directory


In [58]:
#create roll_name_pdf_number pair.
import re

# Path to the  text file
file_path = '/content/driver.txt'

# Initialize lists to store Roll No, Name pairs, and PDF file names
roll_name_pdf_pairs = []

# Read the text file
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Regex patterns for Roll No, Name, and PDF file name
roll_pattern = re.compile(r'Roll No: (\d+)')
name_pattern = re.compile(r'Name: ([A-Za-z\s.]+)')
pdf_pattern = re.compile(r'Attendance_Sheet_VATDE_140_(\d+)_(\d+)\.pdf')

# Temporary variables to store the current Roll No, Name, and PDF file name
current_roll = None
current_name = None
current_pdf = None

r = False
n = False
line_index =0
while line_index < len(lines):
  pdf_match = pdf_pattern.search(lines[line_index])
  roll_match = roll_pattern.search(lines[line_index])
  name_match = name_pattern.search(lines[line_index])
  if pdf_match:
        current_pdf = f'{pdf_match.group(0)}'
        line_index+=1
        continue
  if "Post Name:" in lines[line_index]:
      line_index+=1
      continue
  if roll_match:
    current_roll = roll_match.group(1)
    r=True

    line_index_2 = line_index+1
    while line_index_2 <len(lines):
      roll_match = roll_pattern.search(lines[line_index_2])
      name_match = name_pattern.search(lines[line_index_2])
      if name_match:
        current_name = name_match.group(1)
        n = True
        line_index = line_index_2+1
        break
      if roll_match:
        roll_name_pdf_pairs.append((current_roll, 'null', current_pdf))
        current_roll = roll_match.group(1)
        r = True
        line_index = line_index_2
        break
      line_index_2+=1


  if name_match:
    current_name = name_match.group(1)
    n = True
  if  r and n:
    roll_name_pdf_pairs.append((current_roll, current_name, current_pdf))
    r= False
    n= False
  if not r and n:
    roll_name_pdf_pairs.append(('null', current_name, current_pdf))
    n= False
  line_index+=1

for pair in roll_name_pdf_pairs:
    print(pair)

print(len(roll_name_pdf_pairs))

# i= 15000001
# j=0
# while i< 15001284:
#   roll, name, pdf = roll_name_pdf_pairs[j]
#   if roll=='null':
#     print('x')
#   elif int(roll)!=i:
#     print(f"{roll}, {i}")
#   i+=1
#   j+=1

('14000001', 'MD SHAMIUL BASHAR AKAND\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000002', 'MD NADIM PRODHANIA\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000003', 'MD. ABDULLA\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000004', 'SHAMIM AHMED\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000005', 'MD ALAMIN MIA\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000006', 'MD.SHAMEEM MIA\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000007', 'MD BODRUL SARKER\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000008', 'MD ABDUR RAZZAK\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000009', 'MOJAHID MIA\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000010', 'ZAHIRUL ISLAM BHUIYAN\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000011', 'MD. KHALIL HOSSAIN\n', 'Attendance_Sheet_VATDE_140_14000001_14000048.pdf')
('14000012', 'PARVES\n', 'Attendance_Sheet_VATD

In [59]:

#create pdfname, pagenum, roll count pair
import re

# Path to the uploaded text file
file_path = '/content/driver.txt'

# Initialize a list to store PDF name, page number, and roll count
pdf_page_roll_counts = []

# Read the text file
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Regex patterns for PDF name, Page number, and Roll No
pdf_pattern = re.compile(r'Attendance_Sheet_VATDE_140_(\d+)_(\d+)\.pdf')
name_pattern = re.compile(r'Name: ([A-Za-z\s.]+)')
page_pattern = re.compile(r'Page (\d+)')
roll_pattern = re.compile(r'Roll No: (\d+)')

# Temporary variables to store the current PDF name, page number, and roll count
current_pdf = None
current_page = None
roll_count = 0
pdf = False
page = False



pdf_match = pdf_pattern.search(lines[0])
page_match = page_pattern.search(lines[0])


if pdf_match:
        current_pdf = pdf_match.group(0)
        roll_count = 0
        name_count=0

if page_match:
      current_page = page_match.group(1)

for line in lines:
        if "Post Name:" in line:
          continue
        name_match= name_pattern.search(line)
        roll_match = roll_pattern.search(line)
        if roll_match:
          roll_count+=1
        if name_match:
          name_count+=1

        pdf_match = pdf_pattern.search(line)
        page_match = page_pattern.search(line)
        if pdf_match:
          pdf= True
        if page_match:
          page=True
        if pdf and page:
          pdf_page_roll_counts.append((current_pdf, current_page, max(roll_count, name_count)))
          current_pdf = pdf_match.group(0)

          current_page = page_match.group(1)
          roll_count=0
          name_count=0
          pdf=False
          page= False



if current_pdf is not None and current_page is not None:
    pdf_page_roll_counts.append((current_pdf, current_page, roll_count))
    print(f'Stored: PDF: {current_pdf}, Page: {current_page}, Roll Count: {roll_count}')


for pdf, page, count in pdf_page_roll_counts:
    print(f'PDF: {pdf}, Page: {page}, Roll Count: {count}')

# Print the complete list for debugging
# print(f'Complete pdf_page_roll_counts: {pdf_page_roll_counts}')



Stored: PDF: Attendance_Sheet_VATDE_140_14000241_14000289.pdf, Page: 6, Roll Count: 0
PDF: Attendance_Sheet_VATDE_140_14000001_14000048.pdf, Page: 1, Roll Count: 0
PDF: Attendance_Sheet_VATDE_140_14000001_14000048.pdf, Page: 1, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000001_14000048.pdf, Page: 2, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000001_14000048.pdf, Page: 3, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000001_14000048.pdf, Page: 4, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000001_14000048.pdf, Page: 5, Roll Count: 8
PDF: Attendance_Sheet_VATDE_140_14000049_14000096.pdf, Page: 1, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000049_14000096.pdf, Page: 2, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000049_14000096.pdf, Page: 3, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000049_14000096.pdf, Page: 4, Roll Count: 10
PDF: Attendance_Sheet_VATDE_140_14000049_14000096.pdf, Page: 5, Roll Count: 8
PDF: Attendance_Sheet_VATDE_140_14000097_1400014

In [60]:
pdf_page_roll_counts.pop(0)

('Attendance_Sheet_VATDE_140_14000001_14000048.pdf', '1', 0)

In [20]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 82.6 MB/s eta 0:00:00


In [21]:
pip install pymupdf pillow

In [22]:
pip install pdf2image Pillow opencv-python-headless


In [24]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [23]:
pip install pymupdf opencv-python

In [25]:
#delete a folder
import shutil
dest= '/content/computer'
shutil.rmtree(dest, ignore_errors=True)

In [61]:

#current use extract the faces
from scipy.spatial.distance import euclidean
import os
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image

# Directory containing PDF files
pdf_directory = '/content/drive/MyDrive/লিখিত পরীক্ষার কক্ষ ভিত্তিক হাজিরা শীট/Driver'

# Directory to save the extracted images
image_dir = 'driver'
region_image_dir = 'test'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(region_image_dir, exist_ok=True)

# Output file for detection details
details_file = 'details2.txt'

# Initialize the OCR reader
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Desired size for the resized images
desired_size = (300, 300)
# Padding factor
padding = 20

# Regions for face detection
region_column = [
    (800, 7 , 1300, 1700)
]

region_first_pages = [
    (780, 600, 1300, 1100),
    (750,1050, 1340, 1550)
]

region_three_image =  [

    (792, 0, 1300, 700),
    (792, 570, 1300, 1130),
    (792, 1000, 1300, 1600),

]
regions_of_image=[
    (917,517,1085, 685),
    (917,685, 1085,852),
    (917,852, 1085,1018),
    (917,1018, 1085,1185),
    (917,1185, 1085,1350),
    (917,1350, 1085,1519),
    (917,1519, 1085,1685),
    (917,1685, 1085,1852),
    (917,1852, 1085,2018),
    (917,2018, 1085,2185)
]
region_two_image = [
    (809,11,1393, 600),
    (811,550, 1333,1351)
]

# Initialize the index for accessing roll_name_pairs
index = 0

# Variable to count total number of detected faces
total_faces_detected = 0

# Initialize list to store face counts
imageCount = [0] * (len(roll_name_pdf_pairs))
print(len(imageCount))
# Open the details file to write detection information
with open(details_file, 'w', encoding='utf-8') as details_f:
    # Iterate through each PDF file in the directory
    for pdf_file in sorted(os.listdir(pdf_directory)):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, pdf_file)

            # Convert PDF to a list of images
            pages = convert_from_path(pdf_path)

            # Process each page image
            for page_num, page_image in enumerate(pages):
                # Convert PIL image to OpenCV format
                open_cv_image = np.array(page_image)
                open_cv_image = open_cv_image[:, :, ::-1].copy()

                # Convert to grayscale
                gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)


                for pdf,page, count in pdf_page_roll_counts:
                  if int(page) == int(page_num+1) and pdf== pdf_file:
                    current_roll_count=count


                for j in range (current_roll_count):

                  if index+j < len(roll_name_pdf_pairs):
                          roll, name, pdf_name = roll_name_pdf_pairs[index+j]
                  else:
                          roll, name = 'unknown', 'unknown'
                  rx1, ry1, rx2, ry2 = regions_of_image[j]
                  # if current_roll_count == 2 and page_num!= len(pages)-1 and page_num!= 0:
                  #   rx1, ry1, rx2, ry2 = region_two_image[j]

                  # elif current_roll_count ==2 and page_num == 0:
                  #   rx1, ry1, rx2, ry2 = region_first_pages[j]
                  # else:
                  #   rx1, ry1, rx2, ry2 = region_three_image[j]

                  region_image = open_cv_image[ry1:ry2, rx1:rx2]
                  region_image_gray = gray[ry1:ry2, rx1:rx2]
                  faces = face_cascade.detectMultiScale(region_image_gray, scaleFactor=1.1, minNeighbors=25, minSize=(60, 60))
                  # if page_num == len(pages)-1 and len(faces)==0:
                  #      break
                  details_f.write(f'{pdf_file} - Page Number: {page_num + 1}, roll {roll }, Detected Faces: {len(faces)}\n')

                  #match with the exact regions
                  if len(faces)!=0:
                      total_faces_detected += 1
                      imageCount[index+j] += 1
                      x, y, w, h = faces[0]
                      x1 = max(x + rx1 - padding, 0)
                      y1 = max(y + ry1 - padding, 0)
                      x2 = min(x + w + rx1 + padding, open_cv_image.shape[1])
                      y2 = min(y + h + ry1 + padding, open_cv_image.shape[0])

                      roi = open_cv_image[y1:y2, x1:x2]
                      roi_image = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))

                      # Save the extracted face image

                      image_filename = f'{roll}_{name}.jpg'
                      image_path = os.path.join(image_dir, image_filename)
                      roi_image.save(image_path)
                  else:
                    print(f"No image for {roll}, {name}")
                    print(f" {rx1}, {ry1}, {rx2}, {ry2}")
                    region_image_pil = Image.fromarray(cv2.cvtColor(region_image, cv2.COLOR_BGR2RGB))
                    region_image_filename = f'{roll}_{name}_detection_issue.jpg'
                    region_image_path = os.path.join(image_dir, region_image_filename)
                    region_image_pil.save(region_image_path)


                index+=current_roll_count

print(f'Length of imageCount: {len(imageCount)}')
print(f'Total faces detected: {total_faces_detected}')


289
No image for 14000050, MOHAMMAD ISRAFIL

 917, 685, 1085, 852
No image for 14000062, MD TASRIF HOSSAIN KHAN

 917, 1018, 1085, 1185
No image for 14000082, MD. MONIR HOSSAIN

 917, 1018, 1085, 1185
No image for 14000083, SHAKIL MASUD

 917, 1185, 1085, 1350
No image for 14000141, MD NAJMUL HASAN

 917, 1185, 1085, 1350
No image for 14000150, MD. MIJANUR RAHMAN

 917, 1350, 1085, 1519
No image for 14000221, ARIFUL ISLAM

 917, 1852, 1085, 2018
Length of imageCount: 289
Total faces detected: 282


In [64]:
!zip -r /content/computer.zip /content/computer

updating: content/computer/ (stored 0%)
updating: content/computer/11000017_NEELIMA AFROZ LIMU^J.jpg (deflated 5%)
updating: content/computer/11000071_RAQIBUL UDDIN AHMED^J.jpg (deflated 6%)
updating: content/computer/11000063_MD. ABU BAKER SIDDIQUE TUHIN^J.jpg (deflated 4%)
updating: content/computer/11000020_MD JOSHIM UDDIN^J.jpg (deflated 4%)
updating: content/computer/11000050_FARHANA ALAM MEHTAB^J.jpg (deflated 4%)
updating: content/computer/11000042_LUTFUR NAHAR^J.jpg (deflated 5%)
updating: content/computer/11000049_MD. ISMAIL HOSSAIN RONI^J.jpg (deflated 5%)
updating: content/computer/11000092_AL MAMUN^J.jpg (deflated 5%)
updating: content/computer/11000027_MD. ABU BAKAR JOBAIR^J.jpg (deflated 5%)
updating: content/computer/11000095_MD. SHAHJAHAN^J.jpg (deflated 4%)
updating: content/computer/11000091_MD ASHRAFUL ROBIN^J.jpg (deflated 4%)
updating: content/computer/11000032_AKBAR KHAN^J.jpg (deflated 5%)
updating: content/computer/11000059_SAMIA ISLAM SHARNA^J.jpg (deflated 5%)

In [63]:
#create excel report
import pandas as pd
import numpy as np

# Ensure both lists have the same length
if len(imageCount) > len(roll_name_pdf_pairs):
    # Pad roll_name_pdf_pairs with NaN values
    for _ in range(len(imageCount) - len(roll_name_pdf_pairs)):
        roll_name_pdf_pairs.append(('null', 'null', 'null'))
elif len(roll_name_pdf_pairs) > len(imageCount):
    # Pad imageCount with NaN values
    imageCount.extend([np.nan] * (len(roll_name_pdf_pairs) - len(imageCount)))

# Create DataFrame
df = pd.DataFrame(roll_name_pdf_pairs, columns=['Roll No', 'Name', 'pdf_name'])
df['image_count'] = imageCount

# Save to Excel
excel_path = 'driver.xlsx'
df.to_excel(excel_path, index=False)

print(f'DataFrame saved to {excel_path}')


DataFrame saved to driver.xlsx
